In [59]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
from urllib.parse import urlparse

In [30]:
url_base='https://www.cgmh.org.tw/tw/Systems/'
url_work_table='RecruitInfo/3'
url=url_base+url_work_table
g=requests.get(url)
soup=BeautifulSoup(g.content, 'html.parser')

In [3]:
pages=soup.find_all('ul' ,class_="layout__pagination ul-reset")[0]
def get_pages(pages):
  counter=0
  for i, item in enumerate(pages):
      if item.find('a') :
        s = item.find('a')
        counter += 1
        try:
          #print(s.get("href") ,s.string)
          if s.get("href") == "javascript:void(0)":
            counter -= 1
        except:
          counter -= 1
          #print(item,'error')
      
  return counter
get_pages(pages)


3

In [10]:
pages

<ul class="layout__pagination ul-reset">
<li><a class="active" href="javascript:void(0)">1</a></li><li><a href="/tw/Systems/RecruitInfo/3?page=2#list">2</a></li><li><a href="/tw/Systems/RecruitInfo/3?page=3#list">3</a></li><li><a href="/tw/Systems/RecruitInfo/3?page=2#list"><i class="icon-angle-right"></i></a></li>
</ul>

In [4]:
tables = soup.find('div',class_="bg-grey pd100").find_all('ul')[-2].find_all('li')

In [28]:
def get_work_dead_line(soup):
    work_detail_web = soup.find('article').get_text().replace("\xa0","").replace('\n',"").replace("\u3000","")
    dead_line = work_detail_web.rsplit("報名期限")[1].split("截止")[0].replace("：","")
    return dead_line

In [6]:
work_table=[]

In [62]:
def get_work_page(soup,page):
    if page == 1:
        return soup
    else:
        g=requests.get(url+'?page='+str(page))
        soup=BeautifulSoup(g.content, 'html.parser')
        return soup

In [20]:
def get_work_detail(link):
    g=requests.get(link)
    soup=BeautifulSoup(g.content, 'html.parser')
    return soup    

In [ ]:
def get_work_table(soup,tables,work_table):
    for i, item in enumerate(tables):
        if item.find('a'): #過濾掉被刪除的文章
            s = item.find('a')
            url_base_website = urlparse(url)
            work_detail_link = url_base_website._replace(path=urlparse(s.get('href')).path).geturl()
            title = item.find_all('div')[1].string
            work_page_soup = get_work_detail(work_detail_link)
            dead_line = get_work_dead_line( work_page_soup )
            print('#{}召聘職稱: {} 期限: {}\n 連結：{}'.format(i+1, title, dead_line, work_detail_link ))
            work_table.append([i-2, title, dead_line, work_detail_link ])

In [61]:
for _page in range(1,get_pages(pages)):
    soup_=get_work_page(soup,_page)
    get_work_table(soup_,tables,work_table)

新北市立土城醫院呼吸治療科招募呼吸治療師二名，請 查照。 https://www.cgmh.org.tw/tw/Systems/RecruitContent/3/202204070023
#1召聘職稱: 新北市立土城醫院呼吸治療科招募呼吸治療師二名，請 查照。 期限: 2022年04月20日
 連結：https://www.cgmh.org.tw/tw/Systems/RecruitContent/3/202204070023
林口長庚醫院檢驗醫學部招募抽血技術員，請 查照。 https://www.cgmh.org.tw/tw/Systems/RecruitContent/3/202204070012
#2召聘職稱: 林口長庚醫院檢驗醫學部招募抽血技術員，請 查照。 期限: 2022年04月17日
 連結：https://www.cgmh.org.tw/tw/Systems/RecruitContent/3/202204070012
新北市立土城醫院麻醉科招募麻醉技術員，請 查照。 https://www.cgmh.org.tw/tw/Systems/RecruitContent/3/202204060024
#3召聘職稱: 新北市立土城醫院麻醉科招募麻醉技術員，請 查照。 期限: 2022年04月25日
 連結：https://www.cgmh.org.tw/tw/Systems/RecruitContent/3/202204060024
嘉義長庚紀念醫院招募定期契約戒菸衛教師人員，請 查照。 https://www.cgmh.org.tw/tw/Systems/RecruitContent/3/202204080010
#4召聘職稱: 嘉義長庚紀念醫院招募定期契約戒菸衛教師人員，請 查照。 期限: 2022年4月20日
 連結：https://www.cgmh.org.tw/tw/Systems/RecruitContent/3/202204080010
新北市立土城醫院檢驗醫學科招募定期契約醫事檢驗師，請查照。 https://www.cgmh.org.tw/tw/Systems/RecruitContent/3/202204060059
#5召聘職稱: 新北市立土城醫院檢驗醫學科招募定期契約醫事檢驗師，請查照。 期限: 2022年05月07日
 連結：ht

TypeError: can only concatenate str (not "int") to str

In [ ]:

work_table=pd.DataFrame(work_table, columns=['no','召聘職稱','召聘單位','期限' ,'連結'])

print(work_table)

In [54]:
urlparse(url)

ParseResult(scheme='https', netloc='www.cgmh.org.tw', path='/tw/Systems/RecruitInfo/3', params='', query='', fragment='')